In [1]:
#!/usr/bin/env python
# coding: utf-8
import openai
from openai import AzureOpenAI, AsyncAzureOpenAI
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv, find_dotenv
import requests
import base64


# create ".env" file and put the line with the key in it
# OPENAI_API_KEY="<get_your_key_from_platform.openai.com"
def get_openai_api_key():
    _ = load_dotenv(find_dotenv())

    return os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = get_openai_api_key()
# client = openai.OpenAI(api_key=OPENAI_API_KEY)  # Replace "YOUR_API_KEY" with your actual OpenAI API key

azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

GPT_3_5_TURBO = "gpt-3.5-turbo"
GPT_4_TURBO_PREVIEW = "gpt-4-turbo-preview"
GPT_4 = 'gpt-4'
GPT_4o = 'gpt-4o'

In [5]:
def generate_english_sentences(words, system_prompt, model=GPT_4o):
    word_to_sentence = {}
    for word in tqdm(words, desc="Generating sentece examplesn"):
        #         prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish."
        # prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish. Provide only the Finnish sentence, without any translation or additional information."
        user_prompt = f'''Generate an idiomatic, simple, useful sentence in English containing this word or phrase:
        
        === 
        '{word}'
        ===
         
        The sentence should be useful and easy to understand for someone who just started learning English. Provide only the English sentence, without any translation or additional information.
        '''

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]
        )
        
        sentence = response.choices[0].message.content if response.choices else 'No response generated.'

        word_to_sentence[word] = sentence.strip()
    return word_to_sentence


In [6]:
system_prompt = '''You are generator of simple and usefull sentences.'''
generate_english_sentences(["Apple"], system_prompt, model=GPT_4o)

Generating sentece examplesn:   0%|          | 0/1 [00:00<?, ?it/s]

{'Apple': 'An apple a day keeps the doctor away.'}

In [7]:
def generate_russian_sentences(words, system_prompt, model=GPT_4o):
    word_to_sentence = {}
    for word in tqdm(words, desc="Generating sentece examplesn"):
        #         prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish."
        # prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish. Provide only the Finnish sentence, without any translation or additional information."
        user_prompt = f'''Generate an idiomatic, simple, useful sentence in Russian containing this word or phrase:
        
        === 
        '{word}'
        ===
         
        The sentence should be useful and easy to understand for someone who just started learning Russian. Provide only the Russian sentence, without any translation or additional information.
        '''

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]
        )

        sentence = response.choices[0].message.content if response.choices else 'No response generated.'

        word_to_sentence[word] = sentence.strip()
    return word_to_sentence


In [8]:
system_prompt = '''You are Russian generator of simple and usefull sentences on Russian language.'''
generate_russian_sentences(["Привет"], system_prompt, model=GPT_4o)

Generating sentece examplesn:   0%|          | 0/1 [00:00<?, ?it/s]

{'Привет': 'Привет, как дела?'}

In [9]:
def generate_russian_words(system_prompt, n=1000, model=GPT_4o):
    user_prompt = f'''Generate {n} basic, useful Russian words or phrases for beginners. Provide only generated words or phrases using for separator only comma'''

    response = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
      ]
    )

    sentence = response.choices[0].message.content if response.choices else 'No response generated.'

    return [word.strip() for word in sentence.split(',')]


In [32]:
system_prompt = '''You are the Russian dictionary with useful words and phrases for beginners.'''
russian_basic_words = generate_russian_words(system_prompt, model=GPT_4o)

In [41]:
system_prompt = '''You are Russian generator of simple and usefull sentences on Russian language.'''
russian_word_to_sentence = generate_russian_sentences(russian_basic_words[:100], system_prompt, model=GPT_4o)

Generating sentece examplesn:   0%|          | 0/100 [00:00<?, ?it/s]

In [42]:
russian_word_to_sentence

{'здравствуйте': 'Здравствуйте, как вас зовут?',
 'да': 'Да, я понимаю.',
 'нет': 'Извините, но у меня нет времени.',
 'пожалуйста': 'Пожалуйста, передайте мне соль.',
 'спасибо': 'Спасибо за помощь!',
 'извините': 'Извините, вы не подскажете дорогу?',
 'доброе утро': 'Доброе утро, как вы себя чувствуете сегодня?',
 'добрый день': 'Добрый день, как ваши дела?',
 'добрый вечер': 'Добрый вечер, как у вас дела?',
 'спокойной ночи': 'Спокойной ночи, увидимся завтра!',
 'как дела': 'Как дела? У тебя всё хорошо?',
 'хорошо': 'Всё будет хорошо.',
 'плохо': 'Мне стало плохо.',
 'нормально': 'У меня всё нормально.',
 'отлично': 'Ты сделал это задание отлично!',
 'привет': 'Привет, как дела?',
 'пока': 'Пока, увидимся завтра!',
 'до свидания': 'До свидания, до завтра!',
 'до встречи': 'До встречи на следующей неделе.',
 'до завтра': 'До завтра!',
 'меня зовут': 'Меня зовут Алексей, а тебя?',
 'как вас зовут': 'Как вас зовут, если не секрет?',
 'очень приятно': 'Очень приятно познакомиться!',
 'в

In [43]:
system_prompt = '''You are Russian generator of simple and usefull sentences on Russian language.'''
little_russian_word_to_sentence = generate_russian_sentences(russian_basic_words[:10], system_prompt, model=GPT_4o)

Generating sentece examplesn:   0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
def translate_russian_to_english(word_to_sentence, system_prompt, model=GPT_4o):
    cards = []
    for word, sentence in tqdm(word_to_sentence.items(), desc="Generating sentece examplesn"):
        #         prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish."
        # prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish. Provide only the Finnish sentence, without any translation or additional information."
        #user_prompt = f'''You will get the pair of word or phrase and sentence which contains this word or phrase on Russian then translate it into English. You should translate the word or phrase then translate given sentence into English language.
        #
        #=== 
        #Given word or phrase: '{word}'
        #===
        #Given sentence: '{sentence}'
        #===
        # 
        #The english translated sentence should contain english translated word. Return only english translated word and english translated sentence using comma as seperator and nothing else.
        #'''
        word_prompt = f"""Translate given word or phrase into English.
        ===
        Given word: '{word}'
        ===
        Return only translated word or phrase and nothing else.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": word_prompt}
          ]
        )

        tr_word = response.choices[0].message.content if response.choices else 'No response generated.'
        
        sentence_prompt = f"""Translate given sentence into English. Make sure it contains this word {tr_word}
        ===
        Given sentence: '{sentence}'
        ===
        Return only translated word or phrase and nothing else. Make sentence grammaticly correct and idiomatic.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": sentence_prompt}
          ]
        )

        tr_sentence = response.choices[0].message.content if response.choices else 'No response generated.'
        
        card = {}
        card["wordFirstLang"] = word
        card["sentenceFirstLang"] = sentence
        card["wordSecondLang"] = tr_word.strip()
        card["sentenceSecondLang"] = tr_sentence.strip()
        cards.append(card)
        
    return cards


In [56]:
system_prompt = '''You are the Russian to English translater for beginners. Translate given (word, sentence) pair from Russian to English'''
little_cards = translate_russian_to_english(little_russian_word_to_sentence, system_prompt, model=GPT_4o)

Generating sentece examplesn:   0%|          | 0/10 [00:00<?, ?it/s]

In [57]:
little_cards

[{'wordFirstLang': 'здравствуйте',
  'sentenceFirstLang': 'Здравствуйте, как у вас дела?',
  'wordSecondLang': 'hello',
  'sentenceSecondLang': 'Hello, how are you?'},
 {'wordFirstLang': 'да',
  'sentenceFirstLang': 'Да, я тебя понимаю.',
  'wordSecondLang': 'yes',
  'sentenceSecondLang': 'Yes, I understand you.'},
 {'wordFirstLang': 'нет',
  'sentenceFirstLang': 'Извините, но у нас нет хлеба.',
  'wordSecondLang': 'no',
  'sentenceSecondLang': 'Sorry, but we have no bread.'},
 {'wordFirstLang': 'пожалуйста',
  'sentenceFirstLang': 'Принесите мне стакан воды, пожалуйста.',
  'wordSecondLang': 'please',
  'sentenceSecondLang': 'Bring me a glass of water, please.'},
 {'wordFirstLang': 'спасибо',
  'sentenceFirstLang': 'Спасибо за помощь.',
  'wordSecondLang': 'thank you',
  'sentenceSecondLang': 'Thank you for your help.'},
 {'wordFirstLang': 'извините',
  'sentenceFirstLang': 'Извините, можно мне пройти?',
  'wordSecondLang': 'Sorry',
  'sentenceSecondLang': 'Sorry, can I pass?'},
 {'wo

In [11]:
def eval_cards(cards, first_lang='Russian', second_lang='Finnish', model=GPT_4o):
    successfull_translations = []
    for card in tqdm(cards, desc="Generating sentece examplesn"):
        word_first_lang = card["wordFirstLang"]
        sentence_first_lang = card["sentenceFirstLang"]
        word_second_lang = card["wordSecondLang"]
        sentence_second_lang = card["sentenceSecondLang"]
        
        word_translation_check = f"""Do this {first_lang} word or phrase '{word_first_lang}' is translation of this {second_lang} word or phrase '{word_second_lang}'.
        Provide only 1 if yes, 0 else.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "user", "content": word_translation_check}
          ]
        )

        is_word_translation_correct = response.choices[0].message.content if response.choices else 'No response generated.'
        
        sentence_translation_check = f"""Do this {first_lang} sentence '{sentence_first_lang}' is translation of this {second_lang} sentence '{sentence_second_lang}'.
        Provide only 1 if yes, 0 else.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "user", "content": sentence_translation_check}
          ]
        )

        is_sentence_translation_correct = response.choices[0].message.content if response.choices else 'No response generated.'
        
        first_word_exist_check = f"""Do this {first_lang} word or phrase '{word_first_lang}' contains in this sentence '{sentence_first_lang}'?
        Provide only 1 if yes, 0 else.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "user", "content": first_word_exist_check}
          ]
        )
        
        is_first_word_exists = response.choices[0].message.content if response.choices else 'No response generated.'
        
        second_word_exist_check = f"""Do this {second_lang} word or phrase '{word_second_lang}' contains in this sentence '{sentence_second_lang}'?
        Provide only 1 if yes, 0 else.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "user", "content": second_word_exist_check}
          ]
        )
        
        is_second_word_exists = response.choices[0].message.content if response.choices else 'No response generated.'

        #append succesfull card
        successfull_translations.append(int(is_word_translation_correct) * int(is_sentence_translation_correct) * int(is_first_word_exists) * int(is_second_word_exists))
    return sum(successfull_translations) / len(successfull_translations)


In [64]:
eval_cards(little_cards)

Generating sentece examplesn:   0%|          | 0/10 [00:00<?, ?it/s]

1.0

In [12]:
import json


FOLDER_WITH_JSON = '../../data/russian-finnish/cards/curated_platform_cards/'
def load_cards_from_file(file_name):
    data = []
    file_path = FOLDER_WITH_JSON + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [13]:
file_name = 'sm1_new_kap1.json'
cards = load_cards_from_file(file_name)

In [78]:
eval_cards(cards, first_lang='Russian', second_lang='Finnish')

Generating sentece examplesn:   0%|          | 0/151 [00:00<?, ?it/s]

0.2847682119205298

In [14]:
def evaluate_cards_2(cards, first_lang='Russian', second_lang='Finnish', model=GPT_4o):
    # Construct the prompt
    results = []
    for card in tqdm(cards):
        system_prompt = f"""
        You are a language expert proficient in both {first_lang} and {second_lang}."""

        user_prompt = f"""
        You are given a flashcard containing a word and a sentence in {first_lang}, along with its translation and 
        a sentence in {second_lang}. Your task is to evaluate the correctness of the flashcard. Here’s what you need to check:

        1. Whether the sentence in {first_lang} is grammatically correct and natural.
        2. Whether the sentence in {second_lang} is grammatically correct and natural.
        3. Whether the translation from {second_lang} to {first_lang} is accurate.
        4. Whether the word in {first_lang} occurs in some form in the {first_lang} sentence.
        5. Whether the word in {second_lang} occurs in some form in the {second_lang} sentence.

        If both sentence in {first_lang} and {second_lang} are missing it is ok, just check the translation to {second_lang}. 

        Check that the word in {first_lang} appear in the sentence in some form, diregarding the case and form of the word, or the order of the words, 
        if the words in {first_lang} consists of several words. For example if the word is 'в то же время' and in sentence it's 'в одно и то же время', it's ok. Or if 'когда опять' in the words and 'когда моя сестра опять' is in the sentence. 
        If there are several synonyms in {first_lang} word field, then if one of them appears in the sentences in some form it's ok. 
        For example if {first_lang} word is 'выносить, красть' and in sentence 'украсть' is used it's ok.

        If the word doesn't appear in some form, suggest using synonyms or related terms in the 
        translation to ensure the sentence remains natural and accurate. You can change either the sentence translation 
        or the word translation. For example, if the word is "досуг," it can be translated as "досуг" or "свободное время". So if the word "свободное время" 
        is in some form used in the sentence and it sounds natural, suggested fix can be to change the word translation to "досуг, свободное время", 
        keeping the sentence intact




        In the suggestedFix fields don't provide explanations or instructions, just provide the final corrected string.
        If it's better to fix word, not sentence, return null for suggestedFixSentence.


        Provide a detailed evaluation for each point and suggest fixes where necessary.

        Here is the flashcard:

        Word in {first_lang}: {card['wordFirstLang']}
        Sentence in {first_lang}: {card['sentenceFirstLang']}
        Word in {second_lang}: {card['wordSecondLang']}
        Sentence in {second_lang}: {card['sentenceSecondLang']}

        Respond in JSON format with the following structure:

        {{
          "{first_lang}Sentence": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFix": "Suggested correction if there is an issue"
          }},
          "{second_lang}Sentence": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFix": "Suggested correction if there is an issue"
          }},
          "translationAccuracy": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFix": "Suggested correction if there is an issue"
          }},
          "{first_lang}WordUsage": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFixSentence": "Suggested correction to the sentence if there is an issue"
            "suggestedFixWord": "Suggested correction to the word if there is an issue"
          }},
          "{second_lang}WordUsage": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFix": "Suggested correction if there is an issue"
          }}
        }}


        If both sentence in {first_lang} and {second_lang} are missing return only translationAccuracy block.
        """

        response = client.chat.completions.create(
          model="gpt-4o",
          response_format={ "type": "json_object" },
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]
        )
        
        results.append(json.loads(response.choices[0].message.content.strip()))
    return results

In [92]:
results = evaluate_cards_2(cards, first_lang='Russian', second_lang='Finnish')

  0%|          | 0/151 [00:00<?, ?it/s]

In [93]:
results[0]

{'RussianSentence': {'isCorrect': True,
  'explanation': "The Russian sentence 'Привет, как дела?' is grammatically correct and sounds natural.",
  'suggestedFix': None},
 'FinnishSentence': {'isCorrect': True,
  'explanation': "The Finnish sentence 'Hei, mitä kuuluu?' is grammatically correct and sounds natural.",
  'suggestedFix': None},
 'translationAccuracy': {'isCorrect': True,
  'explanation': "The translation 'Hei, mitä kuuluu?' accurately translates to 'Привет, как дела?' in Russian.",
  'suggestedFix': None},
 'RussianWordUsage': {'isCorrect': True,
  'explanation': "The word 'Привет!' appears in the sentence 'Привет, как дела?' in an appropriate form.",
  'suggestedFixSentence': None,
  'suggestedFixWord': None},
 'FinnishWordUsage': {'isCorrect': True,
  'explanation': "The word 'Hei!' appears in the sentence 'Hei, mitä kuuluu?' in an appropriate form.",
  'suggestedFix': None}}

In [95]:
sum([res['translationAccuracy']['isCorrect'] for res in results]) / len([res['translationAccuracy']['isCorrect'] for res in results])

0.9933774834437086

In [17]:
def translate_first_lang_to_second_lang(initial_cards, first_lang='Russian', second_lang='Finnish', model=GPT_4o):
    final_cards = []
    
    for card in tqdm(initial_cards):
        #         prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish."
        # prompt = f"Generate a simple, useful, beginner-friendly sentence in Finnish using the word or phrase: '{word}'. The sentence should be useful and easy to understand for someone learning Finnish. Provide only the Finnish sentence, without any translation or additional information."
        #user_prompt = f'''You will get the pair of word or phrase and sentence which contains this word or phrase on Russian then translate it into English. You should translate the word or phrase then translate given sentence into English language.
        #
        #=== 
        #Given word or phrase: '{word}'
        #===
        #Given sentence: '{sentence}'
        #===
        # 
        #The english translated sentence should contain english translated word. Return only english translated word and english translated sentence using comma as seperator and nothing else.
        #'''
        
        word_first_lang = card["wordFirstLang"]
        sentence_first_lang = card["sentenceFirstLang"]
        
        word_prompt = f"""Translate given {first_lang} word or phrase into {second_lang}.
        ===
        Given word: '{word_first_lang}'
        ===
        Return only translated word or phrase and nothing else.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": word_prompt}
          ]
        )

        tr_word = response.choices[0].message.content if response.choices else 'No response generated.'
        
        sentence_prompt = f"""Translate given {first_lang} sentence into {second_lang}. Make sure it contains this word {tr_word}
        ===
        Given sentence: '{sentence_first_lang}'
        ===
        Return only translated word or phrase and nothing else. Make sentence grammaticly correct and idiomatic.
        """

        response = client.chat.completions.create(
          model=model,
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": sentence_prompt}
          ]
        )

        tr_sentence = response.choices[0].message.content if response.choices else 'No response generated.'
        
        final_card = {}
        final_card["wordFirstLang"] = word_first_lang
        final_card["sentenceFirstLang"] = sentence_first_lang
        final_card["wordSecondLang"] = tr_word.strip()
        final_card["sentenceSecondLang"] = tr_sentence.strip()
        final_cards.append(card)
        
    return final_cards


In [18]:
translated_cards = translate_first_lang_to_second_lang(cards)

  0%|          | 0/151 [00:00<?, ?it/s]

In [19]:
translation_eval_results = evaluate_cards_2(translated_cards, first_lang='Russian', second_lang='Finnish')

  0%|          | 0/151 [00:00<?, ?it/s]

In [20]:
sum([res['translationAccuracy']['isCorrect'] for res in translation_eval_results]) / len([res['translationAccuracy']['isCorrect'] for res in translation_eval_results])

0.9933774834437086

In [33]:
def eval_with_golden(real_cards, translated_cards, first_lang='Russian', second_lang='Finnish', model=GPT_4o):
    results = []
    for real_card, translated_card in tqdm(list(zip(real_cards, translated_cards))):
        word_real = real_card["wordSecondLang"]
        sentence_real = real_card["sentenceSecondLang"]
        word_tr = translated_card["wordSecondLang"]
        sentence_tr = translated_card["sentenceSecondLang"]
        
        system_prompt = f"""
        You are a language expert proficient in both {second_lang}."""
        
        user_prompt = f"""
        You are given a two pairs of word or phrase and sentence containing this word or phrase in {second_lang}. Here’s what you need to check:

        1. Whether the word or phrase '{word_tr}' is similar by meaning to this word or phrase '{word_real}'.
        2. Whether the sentence '{sentence_tr}' is similar by meaning to this sentence '{sentence_real}'.
        3. Whether the word or phrase '{word_tr}' occurs in some form in this sentence '{sentence_tr}'.

        Respond in JSON format with the following structure:

        {{
          "WordSimilarity": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFix": "Suggested correction if there is an issue"
          }},
          "SentenceSimilarity": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFix": "Suggested correction if there is an issue"
          }},
          "WordUsage": {{
            "isCorrect": true/false,
            "explanation": "Detailed explanation if there is an issue or why it's correct",
            "suggestedFixSentence": "Suggested correction to the sentence if there is an issue"
            "suggestedFixWord": "Suggested correction to the word if there is an issue"
          }}
        }}
        
        If word or sentence didn't provided then think this checks correct
        """
        
        response = client.chat.completions.create(
          model="gpt-4o",
          response_format={ "type": "json_object" },
          messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
          ]
        )
        
        results.append(json.loads(response.choices[0].message.content.strip()))
    return results

In [36]:
results_with_golden = eval_with_golden(cards, translated_cards)

  0%|          | 0/151 [00:00<?, ?it/s]

In [37]:
sum([res['WordSimilarity']['isCorrect'] for res in results_with_golden]) / len([res['WordSimilarity']['isCorrect'] for res in results_with_golden])

0.9933774834437086